In [1]:
import numpy as np
import os
import pickle
from time import time
import lhsmdu
import pdspy.modeling as modeling

In [2]:
param_names = ["Tstar","logLstar","logMdisk","logRdisk","h0","logRin",\
          "gamma","beta","logMenv","logRenv","fcav","ksi","logamax","p","incl"]
ranges = [[3000.,5000.], [-1,3.],[-8.,-2.], [0.,3.],[0.01,0.5], [-1.,2.5], [0.0,1.999], \
          [0.5,2.0],[-8.,-2.],[2.5,4.], [0.,1.], [0.5,1.5], [0.,5.], [2.5,4.5], [0.,90.]]

new_param_names = ["T_star","logL_star","logM_disk","logR_disk","h_0","logR_in",\
               "gamma_trans","bi_x","logM_env_trans","logR_env","f_cav","ksi","loga_max","p","bi_y"]
new_ranges = [[3000.,5000.], [-1,3.],[-8.,-2.], [0.,3.],[0.01,0.5], [-1.,2.5], [np.log10(0.101),np.log10(2.1)], \
          [0,2],[np.log10(0.5),np.log10(6.5)],[2.5,4.], [0.,1.], [0.5,1.5], [0.,5.], [2.5,4.5], [-1.25,0.75]]

In [4]:
# generates coordinates for LHC of "n_models" models, distributed across "ranges"
# returns coordinates by axis and by point.

def generate_lhc(n_models, ranges): 
    dimensions = len(ranges)
    ndsample_01 = np.array(lhsmdu.sample(dimensions, n_models))
    scale = []
    offset = []
    for i in range(len(ranges)):
        scale.append(ranges[i][1]-ranges[i][0])
        offset.append(ranges[i][0])

    ndsample_toscale = ndsample_01 * np.array(scale)[:,np.newaxis] 
    ndsample_inplace = ndsample_toscale + np.array(offset)[:,np.newaxis]

    sample_byaxis = ndsample_inplace
    sample_bypoint = np.transpose(sample_byaxis)
    
    return (sample_byaxis, sample_bypoint)

In [6]:
def orig_to_trans(pars):
    gamma=pars[6]
    logM_env=pars[8]
    beta=pars[7]
    incl=pars[14]
    
    pars[6]=np.log10(2.1-1*gamma)
    pars[8]=np.log10(-1.5-1*logM_env)
    
    s=np.sin(0.7)
    c=np.cos(0.7)
    
    theta=0.7
    pars[7]=1-np.cos(((beta*c) + (incl*s/60)-.5)*np.pi/2)
    pars[14]=(-beta*s) + (incl*c/60)
    return pars

def trans_to_orig(pars):
    gamma_trans=pars[6]
    logM_env_trans=pars[8]
    bi_x=pars[7]
    bi_y=pars[14]
    
    pars[6]=2.1 - 10**gamma_trans
    pars[8]=-1.5 - 10**logM_env_trans
    
    s=np.sin(0.7)
    c=np.cos(0.7)
    
    pars[7]=round((1/(c+(s**2/c)))*((2/np.pi)*np.arccos(1-bi_x)+0.5-(s/c)*bi_y),14)
    pars[14]=round((60*s)*((2/np.pi)*np.arccos(1-bi_x)+0.5+(c/s)*bi_y),14)
    return pars

In [10]:
#t0=time()
#trans_sample=generate_lhc(400,[[0,2],[-1.25,0.75]])
#t1=time()-t0
#print(t1)

#with open('./transsample_byaxis.txt', 'wb') as fp:
#    pickle.dump(trans_sample[0], fp)

#origsample_byaxis=generate_lhc(400,[[0.5,2],[0,90]])[0]
#with open('./origsample_byaxis.txt', 'wb') as fp:
#    pickle.dump(origsample_byaxis, fp)

1223.9470753669739


In [17]:
with open ('./transsample_byaxis.txt', 'rb') as fp:
    transsample_byaxis = pickle.load(fp)
with open ('./origsample_byaxis.txt', 'rb') as fp:
    origsample_byaxis = pickle.load(fp)

In [22]:
def generate_ranges(ranges,scale_factors):
    new_ranges=[]
    for i in range(len(ranges)):
        if scale_factors[i]==1:
            new_ranges.append(ranges[i])
        else:
            a=ranges[i][0]
            b=ranges[i][1]
            s=scale_factors[i]
            new_ranges.append([(a-(s/2-0.5)*(b-a)),(b+(s/2-0.5)*(b-a))])
    return (new_ranges)

In [34]:
scaleup=[4,1.5,1,2,1,1,1,1,1,3,2,1,1,1.8,1]
inflated_ranges=generate_ranges(new_ranges,scaleup)

totalscale=1.5 # for beta/incl cut
for i in range(len(scaleup)):
    totalscale=totalscale*scaleup[i]

print(totalscale*4000)

777600.0


In [ ]:
with open ('./etgrid/etgrid_coords_byaxis_trans.txt', 'rb') as fp:
    coords_byaxis = pickle.load(fp)
with open ('./etgrid/etgrid_coords_bypoint_trans.txt', 'rb') as fp:
    coords_bypoint = pickle.load(fp)
    
with open('./etgrid/etgrid_coords_byaxis_orig.txt', 'wb') as fp:
    pickle.dump(coords_byaxis_orig, fp)
with open('./etgrid/etgrid_coords_bypoint_orig.txt', 'wb') as fp:
    pickle.dump(coords_bypoint_orig, fp)

In [122]:
dictionary=np.load("./gmd/dictionary.npy")


In [1]:
from pyDOE import lhs
from time import time
import numpy as np
import pickle

In [6]:
new_ranges = [[3000.,5000.], [-1,3.],[-8.,-2.], [0.,3.],[0.01,0.5], [-1.,2.5], [np.log10(0.101),np.log10(2.1)], \
          [0,2],[np.log10(0.5),np.log10(6.5)],[2.5,4.], [0.,1.], [0.5,1.5], [0.,5.], [2.5,4.5], [-1.25,0.75]]

In [7]:
def generate_ranges(ranges,scale_factors):
    new_ranges=[]
    for i in range(len(ranges)):
        if scale_factors[i]==1:
            new_ranges.append(ranges[i])
        else:
            a=ranges[i][0]
            b=ranges[i][1]
            s=scale_factors[i]
            new_ranges.append([(a-(s/2-0.5)*(b-a)),(b+(s/2-0.5)*(b-a))])
    return (new_ranges)

scaleup=[4,1.5,1,2,1,1,1,1,1,3,2,1,1,1.8,1]
inflated_ranges=generate_ranges(new_ranges,scaleup)

totalscale=1.5 # for beta/incl cut
for i in range(len(scaleup)):
    totalscale=totalscale*scaleup[i]

In [9]:
n_models=int(4000*totalscale)
ranges=inflated_ranges

dimensions = len(ranges)
ndsample_01 = np.array(lhs(dimensions, samples=n_models))
print("sample successfully drawn")

sample successfully drawn


In [18]:
scale = []
offset = []
for i in range(len(ranges)):
    scale.append(ranges[i][1]-ranges[i][0])
    offset.append(ranges[i][0])

In [32]:
ndsample_toscale = ndsample_01 * np.array(scale)
ndsample_inplace = ndsample_toscale + np.array(offset)

sample_bypoint = ndsample_inplace
sample_byaxis = np.transpose(sample_bypoint)

In [38]:
with open('./etgrid/raw_coords_byaxis.txt', 'wb') as fp:
    pickle.dump(sample_byaxis, fp)
with open('./etgrid/raw_coords_bypoint.txt', 'wb') as fp:
    pickle.dump(sample_bypoint, fp)